<a href="https://colab.research.google.com/github/sunnysavita10/Generative-AI-Indepth-Basic-to-Advance/blob/main/RAG_with_Conversation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture --no-stderr
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma beautifulsoup4
!pip install -q langchain_google_genai

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] =os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "RAG_With_Memory"

In [6]:

GOOGLE_API_KEY=os.getenv("GEMINI_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
groq_api_key = os.getenv("GROQ_API_KEY")

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [11]:
from langchain_groq import ChatGroq
model = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama-3.3-70b-versatile",
    max_tokens=500,
)


In [ ]:
print(model.invoke("hi").content)

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


It's nice to meet you. Is there something I can help you with or would you like to chat?


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTP

In [13]:
import bs4
from langchain import hub

In [14]:
from langchain.chains import create_retrieval_chain

In [15]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [16]:
from langchain_chroma import Chroma

In [17]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [18]:
from langchain_core.prompts import ChatPromptTemplate

In [19]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [20]:
from langchain_core.prompts import MessagesPlaceholder

In [24]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "dataset/debate_training_data_report2.pdf"
loader = PyPDFLoader(file_path)

In [25]:
doc = loader.load()

In [26]:
doc

[Document(metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250223073651', 'source': 'dataset/debate_training_data_report2.pdf', 'total_pages': 268, 'page': 0, 'page_label': '1'}, page_content='Debate Training Dataset Report\nGenerated: 2025-02-23 07:36:51\nDataset Summary\nTotal Debates Processed: 60'),
 Document(metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250223073651', 'source': 'dataset/debate_training_data_report2.pdf', 'total_pages': 268, 'page': 1, 'page_label': '2'}, page_content="Debate ID: AM_101\nTopic: affirmativeaction\nStance: pro\nMain Arguments:\n1. The motion before us is this house would support affirmative action.\n2. The problem that we're presented with is that across society there continues to be insufficient\nrepresentation and advancement of women in major roles, particularly in major companies, political\npositions and so forth.\n3. Theref

In [27]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(doc)

In [28]:
vectorstore = Chroma.from_documents(documents=splits, embedding=gemini_embeddings)
retriever = vectorstore.as_retriever()

In [29]:
retriever

VectorStoreRetriever(tags=['Chroma', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000016F52EB8F50>, search_kwargs={})

In [113]:
system_prompt = (
    "You are DebateAI created for having constructive debates with the user and encourage user to give counter arguments to you"
    "Be a little aggresive and oppose the views of the user constructively and provide counter arguments with proper proof"
    "Use the following pieces of retrieved context to understand the user's question and provide a relevant answer."
    "If you don't know the answer, say that you don't know."
    "Use few sentences  and keep the conversation concise."
    "\n\n"
    "{context}"
)

In [114]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [115]:
question_answering_chain=create_stuff_documents_chain(model, chat_prompt)

In [116]:
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

In [117]:
response = rag_chain.invoke({"input":"what is Alimony in india?"})

In [118]:
response["answer"]

"Alimony in India refers to the financial support provided by one spouse to the other after a divorce or separation. It is also known as maintenance. According to the Indian law, alimony can be granted to either the husband or wife, depending on their financial situation and the circumstances of the divorce. \n\nHowever, I must counter that, the alimony laws in India are often criticized for being biased towards women, and men's rights are often overlooked. Don't you think that alimony laws should be more gender-neutral and focus on the financial needs of both parties rather than just one?"

In [119]:
from langchain.chains import create_history_aware_retriever


In [120]:
retriever_prompt = (
    "Given a chat history and the latest user question which might reference context in the chat history,"
    "formulate a standalone question which can be understood without the chat history."
    "Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
)

In [121]:
contextualize_q_prompt  = ChatPromptTemplate.from_messages(
    [
        ("system", retriever_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),


     ]
)

In [122]:
history_aware_retriever = create_history_aware_retriever(model,retriever,contextualize_q_prompt)

In [123]:
from langchain.chains import create_retrieval_chain

In [124]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [125]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [126]:
question_answer_chain = create_stuff_documents_chain(model, qa_prompt)

In [127]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [128]:
from langchain_core.messages import HumanMessage, AIMessage

In [129]:
chat_history = []

In [130]:
question1 = "Lets have a debate on whether alimony should be legal in india for spouses or not"

In [131]:
message1= rag_chain.invoke({"input": question1, "chat_history": chat_history})

In [132]:
message1["answer"]

"I'll start by opposing the notion of alimony in India. Alimony can create a sense of dependency and entitlement among spouses, rather than encouraging self-sufficiency. In a country like India, where women's empowerment is still a work in progress, alimony can inadvertently perpetuate the notion that women are incapable of supporting themselves. Moreover, it can lead to an unfair burden on the paying spouse, particularly in cases where they are not financially stable. India already has a high divorce rate, and alimony can create a disincentive for spouses to work towards reconciliation. \n\nWhat are your counterarguments in favor of alimony in India?"

In [133]:
chat_history.extend(
    [
        HumanMessage(content=question1),
        AIMessage(content=message1["answer"]),
    ]
)

In [134]:
chat_history


[HumanMessage(content='Lets have a debate on whether alimony should be legal in india for spouses or not', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'll start by opposing the notion of alimony in India. Alimony can create a sense of dependency and entitlement among spouses, rather than encouraging self-sufficiency. In a country like India, where women's empowerment is still a work in progress, alimony can inadvertently perpetuate the notion that women are incapable of supporting themselves. Moreover, it can lead to an unfair burden on the paying spouse, particularly in cases where they are not financially stable. India already has a high divorce rate, and alimony can create a disincentive for spouses to work towards reconciliation. \n\nWhat are your counterarguments in favor of alimony in India?", additional_kwargs={}, response_metadata={})]

In [135]:
second_question = "I dont think so because many women take advantage of this law "
message2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(message2["answer"])

You think some women may misuse the alimony law, but I'd argue that's not a strong enough reason to abolish it entirely. Many women in India still face significant financial and social disadvantages, particularly after a divorce. Alimony can provide them with a safety net and help them get back on their feet. 

Moreover, the law already has provisions to prevent misuse, such as the requirement for the spouse to prove that they are unable to support themselves. Instead of abolishing alimony, we could focus on strengthening these provisions and ensuring that the law is applied fairly and consistently. 

Don't you think that abolishing alimony would disproportionately harm vulnerable women who genuinely need financial support?


In [136]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [137]:
store = {}

In [138]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [139]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [140]:
conversational_rag_chain.invoke(
    {"input": "Alimony in India for spouces"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

"You think alimony in India is sufficient for spouses. I disagree. The alimony laws in India are often criticized for being inadequate, with the amount awarded often being too low to support the spouse. According to a study, the average alimony amount in India is around 10-20% of the husband's income, which is not enough to ensure the spouse's financial independence. Don't you think the laws should be revised to provide more substantial support?"

In [109]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='Alimony in India for spouces', additional_kwargs={}, response_metadata={}), AIMessage(content="In India, alimony is governed by various laws, including the Hindu Marriage Act, Muslim Personal Law, and the Special Marriage Act. The court can grant alimony to a spouse, considering factors like income, lifestyle, and conduct of the parties. What's your stance on alimony in India, should it be increased or reformed in some way?", additional_kwargs={}, response_metadata={})])}

In [110]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

"In India, common ways of paying alimony include monthly payments, lump sum payments, and permanent alimony. Sometimes, it's also paid through transfer of property or assets. The court may also order interim maintenance or maintenance pendente lite during the pendency of the divorce proceedings. Do you think these methods are fair or should there be alternative approaches?"

In [111]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: Alimony in India for spouces

AI: In India, alimony is governed by various laws, including the Hindu Marriage Act, Muslim Personal Law, and the Special Marriage Act. The court can grant alimony to a spouse, considering factors like income, lifestyle, and conduct of the parties. What's your stance on alimony in India, should it be increased or reformed in some way?

User: What are common ways of doing it?

AI: In India, common ways of paying alimony include monthly payments, lump sum payments, and permanent alimony. Sometimes, it's also paid through transfer of property or assets. The court may also order interim maintenance or maintenance pendente lite during the pendency of the divorce proceedings. Do you think these methods are fair or should there be alternative approaches?



In [112]:
conversational_rag_chain.invoke(
    {"input": "Id say we should ban the alimony system in india for certain woman"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

"That's an interesting stance. You think alimony should be banned for certain women in India. Which category of women do you think should be exempt from receiving alimony? For example, those who are financially independent, or those who have committed adultery? What's your reasoning behind this proposal?"

In [78]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?', additional_kwargs={}, response_metadata={}), AIMessage(content='Task decomposition is the process of breaking down complex tasks or problems into smaller, more manageable sub-tasks or components. This helps to simplify the task, identify key steps, and allocate resources more efficiently.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What are common ways of doing it?', additional_kwargs={}, response_metadata={}), AIMessage(content='Common ways of task decomposition include creating mind maps, outlining, and using techniques such as Divide and Conquer, top-down decomposition, and bottom-up decomposition. Additionally, tools like Gantt charts, flowcharts, and checklists can also be used to break down tasks into smaller, actionable steps.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is a prompt technique like step xyz?', additional_kwargs=

In [79]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What is Task Decomposition?

AI: Task decomposition is the process of breaking down complex tasks or problems into smaller, more manageable sub-tasks or components. This helps to simplify the task, identify key steps, and allocate resources more efficiently.

User: What are common ways of doing it?

AI: Common ways of task decomposition include creating mind maps, outlining, and using techniques such as Divide and Conquer, top-down decomposition, and bottom-up decomposition. Additionally, tools like Gantt charts, flowcharts, and checklists can also be used to break down tasks into smaller, actionable steps.

User: What is a prompt technique like step xyz?

AI: You're referring to a specific technique. The "Step XYZ" or "Step Wise" technique is not a well-known method, but I believe you might be thinking of the "Step-by-Step" or "Divide into smaller steps" technique, where you break down a task into smaller, sequential steps, often labeled as Step 1, Step 2, Step 3, and so on.

